In [14]:
%%capture
!pip install prophet

In [15]:
from datetime import datetime, timedelta
import psycopg2
from prophet import Prophet
import pandas as pd
import numpy as np
import sys

In [17]:
#?Prophet

# Prophet(
#     growth='linear',
#     changepoints=None,
#     n_changepoints=25,
#     changepoint_range=0.8,
#     yearly_seasonality='auto',
#     weekly_seasonality='auto',
#     daily_seasonality='auto',
#     holidays=None,
#     seasonality_mode='additive',
#     seasonality_prior_scale=10.0,
#     holidays_prior_scale=10.0,
#     changepoint_prior_scale=0.05,
#     mcmc_samples=0,
#     interval_width=0.8,
#     uncertainty_samples=1000,
#     stan_backend=None,
#     scaling: str = 'absmax',
#     holidays_mode = None
# )

In [20]:
# Выгружаем из базы данных
# Конфигурация
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'server_metrics',
    'user': 'postgres',
    'password': 'postgres'
}

In [21]:
def db_export(vm, metric, start_date, end_date):
    """
    Выгрузка данных за период времени

    Args:
        vm: сервер
        metric: метрика
        start_date: выгрузка С
        end_date: выгрузка ПО
    """

    try:
        # Подключение к базе
        conn = psycopg2.connect(**DB_CONFIG)

        # SQL запрос
        query = """
        SELECT
            timestamp,
            value
        FROM server_metrics_fact
        WHERE
            vm = %s
            AND metric = %s
            AND timestamp BETWEEN %s AND %s
        ORDER BY timestamp ASC
        """

        # Выполнение запроса
        df = pd.read_sql_query(query, conn, params=(vm, metric, start_date, end_date))

        # Закрытие соединения
        conn.close()

        if df.empty:
            print(f"Данные не найдены для {vm} - {metric}")
            return None

        # Вывод информации
        print(f"Выгружено {len(df)} записей")
        print(f"Период: {df['timestamp'].min()} - {df['timestamp'].max()}")
        print(f"Значения: {df['value'].min():.2f}% - {df['value'].max():.2f}%")
        print(f"Среднее: {df['value'].mean():.2f}%")

        return df

    except Exception as e:
        print(f"Ошибка: {e}")
        return None